In [ ]:
import pandas as pd

df = pd.read_csv('/Users/TN76JP/Desktop/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_IX,+all.csv')

df

In [ ]:
df_clean=df[["REF_AREA","TIME_PERIOD","OBS_VALUE","CURRENCY","UNIT_MEASURE","Measure"]]

df_clean=df_clean[df_clean["Measure"].isin(["Exchange rates, annual average"])]

df_clean=df_clean[df_clean["TIME_PERIOD"].isin([2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024])]

df_clean

In [ ]:
import classifications

iso3=classifications.location.datapackage.tree_iso_3166_1_alpha3.copy()
df_clean=df_clean[df_clean["REF_AREA"].isin(iso3["code"])]
df_clean

In [ ]:
curr=classifications.unit_monetary.datapackage.bonsai_currency_to_location.copy()
df_clean=df_clean[df_clean["CURRENCY"].isin(curr["currency"])]
df_clean

In [ ]:
df_clean["unit"] = df_clean.apply(
    lambda row: row["UNIT_MEASURE"].replace("XDC_", row["CURRENCY"] + "/")
    if "XDC_" in row["UNIT_MEASURE"] else row["UNIT_MEASURE"],
    axis=1
)
df_clean

In [ ]:
df_clean=df_clean.rename(columns={"REF_AREA":"location", "TIME_PERIOD":"time","OBS_VALUE":"value"})
df_clean

In [ ]:
df_clean=df_clean[["location","time","value","unit"]]
df_clean

In [ ]:
# Filter rows with unit == "EUR/USD"
eur_usd_rows = df_clean[df_clean["unit"] == "EUR/USD"].copy()

# Create new rows
new_rows = eur_usd_rows.copy()
new_rows["value"] = new_rows["value"] / 1_000_000
new_rows["unit"] = "Meuro/USD"

# Combine original df and new rows
df_extended = pd.concat([df_clean, new_rows], ignore_index=True)
df_extended

In [ ]:
df_extended.to_csv("/Users/TN76JP/Documents/bonsai/lookup/src/classifications/data/unit/monetary/fact_currency_per_usd.csv",index=False)